# 03 — Closed-loop (propose → fold → select → mutate)

In [ ]:
!pip -q install transformers accelerate biopython pandas numpy matplotlib tqdm scikit-learn pyyaml
import torch, platform
print("torch", torch.__version__, "cuda?", torch.cuda.is_available(), "python", platform.python_version())


In [ ]:
!git clone -q https://github.com/dauparas/ProteinMPNN.git
!pip -q install -r ProteinMPNN/requirements.txt


In [ ]:
from pathlib import Path
from src.data.scaffolds import load_scaffold
from src.generate.proteinmpnn import run_proteinmpnn, read_fasta_sequences
from src.generate.mutations import make_mutant_pool
from src.evaluate.esmfold_eval import evaluate_batch
from src.loop.closed_loop import Candidate, run_closed_loop
import pandas as pd

OUT = Path("results")
PDB_ID, CHAIN = "1AKL","A"
sc = load_scaffold(PDB_ID, CHAIN, OUT/"scaffolds")
seed_seq = sc.sequence[:250]

def propose_fn(seeds, n, r):
    if r == 0:
        fasta = run_proteinmpnn(sc.pdb_path, OUT/"mpnn_round0", Path("ProteinMPNN"), num_seqs=n, sampling_temp=0.2, seed=42)
        return read_fasta_sequences(fasta)[:n]
    return make_mutant_pool(seeds, n=n, rate=0.03, seed=42+r)

def eval_fn(seqs, r):
    res = evaluate_batch(seqs, model_id="facebook/esmfold_v1", device="cuda", out_dir=OUT/"pdb"/f"round_{r:02d}", max_n=25)
    return [Candidate(sequence=x.sequence, score=x.mean_plddt) for x in res]

df, best = run_closed_loop([seed_seq], propose_fn, eval_fn, rounds=4, per_round=50, top_k=10)
df


In [ ]:
from pathlib import Path
from src.metrics.figures import plot_round_curves
OUT = Path("results")
(OUT/"tables").mkdir(parents=True, exist_ok=True)
df.to_csv(OUT/"tables"/"closed_loop.csv", index=False)
plot_round_curves(df, OUT/"figures"/"closed_loop_round_curves.png")
print("saved tables/figures in", OUT)
